In [1]:
import os
import scipy
import numpy as np
import pylab as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib import pyplot as mp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.io import savemat, loadmat
from sklearn import datasets, linear_model, preprocessing
from scipy.stats import probplot, pearsonr
from sklearn.preprocessing import scale,robust_scale
from sklearn.metrics import mean_squared_error, r2_score
from scipy import interp
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

/Users/emilyhahn/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Configure environment

In [2]:
dACC_group=[]
L_dlPFC_group=[]
R_dlPFC_group=[]
L_IFG_group=[]
R_IFG_group=[]
conflict_group=[]
adapt_group=[]
rt_group=[]

dACC_group_hc=[]
L_dlPFC_group_hc=[]
R_dlPFC_group_hc=[]
L_IFG_group_hc=[]
R_IFG_group_hc=[]
conflict_group_hc=[]
adapt_group_hc=[]
rt_group_hc=[]

dACC_group_pts=[]
L_dlPFC_group_pts=[]
R_dlPFC_group_pts=[]
L_IFG_group_pts=[]
R_IFG_group_pts=[]
conflict_group_pts=[]
adapt_group_pts=[]
rt_group_pts=[]

regions=['dACC','L_dlPFC','R_dlPFC','L_IFG','R_IFG']
variables=['rt','adapt','conflict']
raw_behav_dir='/Users/emilyhahn/projects/msit_modeling/behavior_preproc/msit_mri_behav'
preproc_behav_dir='/Users/emilyhahn/projects/msit_modeling/behavior_preproc/completed'
LSS_dir='/Users/emilyhahn/projects/msit_modeling/LSS_AVG'
LSS_estim_dir='/Users/emilyhahn/projects/msit_modeling/LSS_AVG_estim'
censor_dir='/Users/emilyhahn/projects/msit_modeling/censor_data'

df=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_I=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_C=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_I_C=pd.DataFrame({"group":[],"subject":[],'rt':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'acc':[],'conflict':[],'adapt':[]})

df_I_epoch_first45=pd.DataFrame({"group":[],"subject":[],'rt':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'acc':[],'conflict':[],'adapt':[]})
df_C_epoch_first45=pd.DataFrame({"group":[],"subject":[],'rt':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'acc':[],'conflict':[],'adapt':[]})

### Define Subjects

In [3]:
# SUBJ_LIST = ['hc001','hc002','hc003','hc005','hc006','hc010','hc011','hc012','hc014',\
#              'hc015','hc017','hc019','hc021','hc028','hc031','hc032','hc033','hc034',\
#              'hc036','hc038','hc042','pp001','pp002','pp003','pp004','pp005','pp006',\
#              'pp010','pp011','pp012','pp013','pp015','pp016']
SUBJ_LIST = ['hc001']

### Define Functions - Data Structs

In [4]:
total_outliers=[]
def remove_outliers(rt_var,RT_arr,sub):
    elements=[i for i in RT_arr]
    mean = np.nanmean(elements)
    sd = np.nanstd(elements)
    sd_lower = mean - 5 * sd
    sd_upper = mean + 5 * sd
    for x in elements:
        if (sd_lower <= x <= sd_upper):
            rt_var.append(x)
        else:
            rt_var.append('NaN')
            print '%s\n%.2f RT value excluded: not in range SD min (%.2f) to SD max (%.2f)' % (sub,x,sd_lower,sd_upper)
    return

def generate_roi_vars(roi_file,region,var,sub):
    elements=[]
    with open(roi_file, 'r') as file:
        for line in file:
            line=line.strip()
            line=float(line)
            elements.append(line)
    mean = np.nanmean(elements)
    sd = np.nanstd(elements)
    sd_lower = mean - 5 * sd
    sd_upper = mean + 5 * sd
    for x in elements:
        if (sd_lower <= x <= sd_upper):
            region.append(x)
        else:
            region.append('NaN')
            total_outliers.append(x)
            print '%s\n%.2f %s beta excluded: not in range SD min (%.2f) to SD max (%.2f)' % (sub,x,var,sd_lower,sd_upper) 
    return

def censor_tps(censor_file,censor_var,var):
    with open(censor_file, 'r') as file:
        for line in file:
            line=line.strip()
            censor_var.append(line)
    return

def build_plots(ROI,variable,df_type,group,cond,sub):
    slope, intercept, r_value, p_value, std_err = \
            stats.linregress(df_type['%s' % variable],df_type['%s' % ROI])
#     if (p_value <= 1):
    sns.set()
    g=sns.JointGrid(x='%s' % variable, y='%s' % ROI, data=df_type)
    g=g.plot(sns.regplot,sns.distplot)
    lin_reg_r = lambda a, b: stats.linregress(df_type['%s' % variable], df_type['%s' % ROI])[2:4]
    g = g.annotate(lin_reg_r, template="{stat}: {val:.2f} $p=$ {p:.2f}",stat="$r=$",loc="upper left")                                                                        
    plt.subplots_adjust(top=0.93)
#     g.fig.suptitle("%s | %s" % (group,cond))
    g.fig.suptitle("%s | %s | %s" % (group,cond,sub))
#     g.savefig('/Users/emilyhahn/projects/msit_modeling/figures/%s_%s_%s_%s.png' % (ROI,variable,group,cond))
    plt.show()
#     else:
#         print "**** %s %s %s not significant ****" % (SUBJ,ROI,cond)
    return

### Assemble Indiv and Group Data - SS output, ROI data
    This module generates DataFrames:
    df = ALL subjects, ALL trials
    df_I = ALL subjects, Incongruent trials
    df_C = ALL subjects, Congruent trials
    df_I_C =ALL subjects, Incongruent-Congruent trials
    df_hcs = CTRL subjects, ALL trials
    df_pts = PSYCH subjects, ALL trials
    df_hcs_I = CTRL subjects, Incongruent trials
    df_pts_I = PSYCH subjects, Incongruent trials
    df_hcs_C = CTRL subjects, Congruent trials
    df_pts_C = PSYCH subjects, Congruent trials

In [24]:
for SUBJ in SUBJ_LIST:
    rt=[]
    RT=[]
    RT_orig=[]
    conflict=[]
    adapt=[]
    dACC=[]
    L_IFG=[]
    R_IFG=[]
    L_dlPFC=[]
    R_dlPFC=[]
    cond=[]
    trial=[]
    acc=[]
    censor=[]        
    #---------------------------------##
    ## Configure SS variables
    #---------------------------------##
    mat = loadmat(os.path.join(preproc_behav_dir,'%s_msit_ss_iter250.mat') % SUBJ)
    #---------------------------------##
    ss_outputs_xsmt = np.expand_dims(np.array([np.concatenate(arr) \
                                for arr in mat['XSmt']]).squeeze(),1)
    ss_xsmt = ss_outputs_xsmt.squeeze()
    RT.extend([float(i) for i in np.array(mat['RT'].squeeze())])
    remove_outliers(rt,RT,SUBJ)
    #---------------------------------##
    conflict.extend([float(i[0]) for i in ss_xsmt])
    adapt.extend([float(i[1]) for i in ss_xsmt])
    acc.extend([float(i) for i in np.array(mat['Accuracy'].squeeze())])
    trial.extend([float(i) for i in np.array(mat['Trial'].squeeze())])
    cond.extend([float(i) for i in np.array(mat['Interference'].squeeze())])
    #---------------------------------##
    ## Configure ROI variables
    #---------------------------------##
    ## Load indiv ROI vars
    file_1=os.path.join(LSS_dir,'%s.dACC_LSS_avg_file.1D' % SUBJ)
    file_2=os.path.join(LSS_dir,'%s.L_IFG_LSS_avg_file.1D' % SUBJ)
    file_3=os.path.join(LSS_dir,'%s.R_IFG_LSS_avg_file.1D' % SUBJ)
    file_4=os.path.join(LSS_dir,'%s.L_dlPFC_LSS_avg_file.1D' % SUBJ)
    file_5=os.path.join(LSS_dir,'%s.R_dlPFC_LSS_avg_file.1D' % SUBJ)
    file_6=os.path.join(censor_dir,'msit.%s.msit_bsm.censor.1D' % SUBJ)
    #---------------------------------##
    ## Generate ROI variables
    generate_roi_vars(file_1,dACC,"dACC",SUBJ)
    generate_roi_vars(file_2,L_IFG,"L_IFG",SUBJ)
    generate_roi_vars(file_3,R_IFG,"R_IFG",SUBJ)
    generate_roi_vars(file_4,L_dlPFC,"L_dlPFC",SUBJ)
    generate_roi_vars(file_5,R_dlPFC,"R_dlPFC",SUBJ)
    #---------------------------------##
    censor_tps(file_6,censor,"censor")
    #---------------------------------##
    ## insert RT outliers scrubbing function here
    #---------------------------------##
    ## Determine group
    s=[]
    if 'hc' in SUBJ:
        s.append(0)
    elif 'pp' in SUBJ:
        s.append(1)
    GROUP_ARR=np.array(["%s" % s] * len(trial))
    group=[float(i.strip('[]')) for i in GROUP_ARR]
    SUBJ_ARR=np.array(["%s" % SUBJ] * len(trial))
    #---------------------------------## 
    ## Scale + interpolate missing ROI + RT data points
    ## Configure this section to statistically appropriate scaling
    
    dACC = pd.Series(dACC).astype(float).interpolate()
    R_IFG = pd.Series(R_IFG).astype(float).interpolate()
    L_IFG = pd.Series(L_IFG).astype(float).interpolate()
    L_dlPFC = pd.Series(L_dlPFC).astype(float).interpolate()
    R_dlPFC = pd.Series(R_dlPFC).astype(float).interpolate()

    rt = preprocessing.scale(pd.Series(rt).astype(float).interpolate())
    #---------------------------------## 
    dACC_group.append(dACC)
    L_dlPFC_group.append(L_dlPFC)
    R_dlPFC_group.append(R_dlPFC)
    L_IFG_group.append(L_IFG)
    R_IFG_group.append(R_IFG)
    conflict_group.append(conflict)
    adapt_group.append(adapt)
    rt_group.append(rt)
    #---------------------------------##
    if 'pp' in SUBJ_ARR[0]:
        dACC_group_pts.append(dACC)
        L_dlPFC_group_pts.append(L_dlPFC)
        R_dlPFC_group_pts.append(R_dlPFC)
        L_IFG_group_pts.append(L_IFG)
        R_IFG_group_pts.append(R_IFG)
        conflict_group_pts.append(conflict)
        adapt_group_pts.append(adapt)
        rt_group_pts.append(rt)
    elif 'hc' in SUBJ_ARR[0]:
        dACC_group_hc.append(dACC)
        L_dlPFC_group_hc.append(L_dlPFC)
        R_dlPFC_group_hc.append(R_dlPFC)
        L_IFG_group_hc.append(L_IFG)
        R_IFG_group_hc.append(R_IFG)
        conflict_group_hc.append(conflict)
        adapt_group_hc.append(adapt)  
        rt_group_hc.append(rt)
    else:
        "REVIEW SUBJ ID: %s" % SUBJ
    ##---------------------------------##
    ## Append subj data to master DF
    ##---------------------------------##
    df1=pd.DataFrame({"group":group,"subject":SUBJ_ARR,'rt':rt,'dACC':dACC,'L_IFG':L_IFG,\
                      'R_IFG':R_IFG,'L_dlPFC':L_dlPFC,'R_dlPFC':R_dlPFC,'trial':trial,\
                      'cond':cond,'acc':acc,'conflict':conflict,'adapt':adapt})
    ##---------------------------------##
    ## Determine AIC, BIC
    ##---------------------------------##
    def OLS(equation,df_type,df_name):
        t_model = smf.ols(formula = '%s' % equation, data=df_type)
        t_lin_reg = t_model.fit()
        pvals = t_lin_reg.pvalues
        coeff = t_lin_reg.params
        stderr = t_lin_reg.bse
        AIC = t_lin_reg.aic
        conf_lower = t_lin_reg.conf_int(alpha=0.05,cols=None)[0]
        conf_higher = t_lin_reg.conf_int(alpha=0.05,cols=None)[1]
        results_df = pd.DataFrame(data={'model':equation,"subj":SUBJ,"p_vals":pvals,
                                   "β_coef":coeff,"std_err":stderr,"CI_low":conf_lower,
                                    "CI_high":conf_higher,"AIC":AIC})
        results_df.index.name = 'variable'
        results_df.reset_index(level=0, inplace=True)
        results_df = results_df[["subj","model","variable","AIC","β_coef","p_vals","std_err","CI_low","CI_high"]]  
        display(results_df)
        return 
    ##---------------------------------##
    for cog_roi in regions:
        OLS('%s ~ 1' % cog_roi,df1,'%s' % SUBJ)
        OLS('%s ~ rt + 1' % cog_roi,df1,'%s' % SUBJ)
        OLS('%s ~ adapt + 1' % cog_roi,df1,'%s' % SUBJ)
        OLS('%s ~ conflict + 1' % cog_roi,df1,'%s' % SUBJ)
        OLS('%s ~ adapt + conflict + 1' % cog_roi,df1,'%s' % SUBJ)
    ##---------------------------------##
    ##---------------------------------##
    df=df.append(df1)
    df1_C=df1[df1.cond == 0]
    df1_I=df1[df1.cond == 1]
    ##---------------------------------##
#     conditions = ['Incongruent','Congruent']
#     rois = ['dACC']

#     for c in conditions:
#         for region in rois:
#             for variable in variables:
#                 if 'pp' in SUBJ and c=='Incongruent':
#                     build_plots('%s' % region,'%s' % variable,df1_I,'PSYCH','%s' % c,SUBJ)
#                 elif 'pp' in SUBJ and c=='Congruent':
#                     build_plots('%s' % region,'%s' % variable,df1_C,'PSYCH','%s' % c,SUBJ)
#                 elif 'hc' in SUBJ and c=='Incongruent':
#                     build_plots('%s' % region,'%s' % variable,df1_I,'CTRL','%s' % c,SUBJ)
#                 elif 'hc' in SUBJ and c=='Congruent':
#                     build_plots('%s' % region,'%s' % variable,df1_C,'CTRL','%s' % c,SUBJ)
#                 else:
#                     print '%s %s %s %s' % (g,c,region,variable)
    ##---------------------------------##
    df1_C_epoch_first45=df1_C[:][0:50].reset_index(drop=True).mean(axis=0,numeric_only=True) 
    df1_I_epoch_first45=df1_I[:][0:50].reset_index(drop=True).mean(axis=0,numeric_only=True)
    ##---------------------------------##
    df1_C=df1_C.mean(axis=0,numeric_only=True)
    df1_I=df1_I.mean(axis=0,numeric_only=True)
    ##---------------------------------##
    ##---------------------------------##
    S=group[0]
    df1_C['subject']=SUBJ
    df1_C['group']=S
    df1_I['subject']=SUBJ
    df1_I['group']=S
    df1_I_epoch_first45['subject']=SUBJ
    df1_I_epoch_first45['group']=S
    df1_C_epoch_first45['subject']=SUBJ
    df1_C_epoch_first45['group']=S
    ##---------------------------------##
    df_I=df_I.append(df1_I,ignore_index=True)
    df_C=df_C.append(df1_C,ignore_index=True)
    
    df_I_epoch_first45=df_I_epoch_first45.append(df1_I_epoch_first45,ignore_index=True)
    df_C_epoch_first45=df_C_epoch_first45.append(df1_C_epoch_first45,ignore_index=True)
    ##---------------------------------##
    df1_I_C=pd.DataFrame(data={"group":S,"subject":SUBJ,'rt':df1_I['rt']-df1_C['rt']},index=[1])
    df1_I_C={"group":S,"subject":SUBJ,\
             'rt':df1_I['rt']-df1_C['rt'],\
             'dACC':df1_I['dACC']-df1_C['dACC'],\
             'L_IFG': df1_I['L_IFG']-df1_C['L_IFG'],\
             'R_IFG': df1_I['R_IFG']-df1_C['R_IFG'],\
             'L_dlPFC': df1_I['L_dlPFC']-df1_C['L_dlPFC'],\
             'R_dlPFC': df1_I['R_dlPFC']-df1_C['R_dlPFC'],\
             'acc': df1_I['acc']-df1_C['acc'],\
             'conflict': df1_I['conflict']-df1_C['conflict'],\
             'adapt': df1_I['adapt']-df1_C['adapt'] }
    df_I_C=df_I_C.append(df1_I_C,ignore_index=True)
##---------------------------------##
## Parse DataFrames
##---------------------------------##
df_hcs=df.where(df['group']==0).dropna()
df_pts=df.where(df['group']==1).dropna()
df_hcs_I=df_I.where(df_I['group']==0).dropna().reset_index(drop=True)
df_pts_I=df_I.where(df_I['group']==1).dropna().reset_index(drop=True)
df_hcs_C=df_C.where(df_C['group']==0).dropna().reset_index(drop=True)
df_pts_C=df_C.where(df_C['group']==1).dropna().reset_index(drop=True)
df_hcs_I_epoch_first45=df_I_epoch_first45.where(df_I_epoch_first45['group']==0).dropna().reset_index(drop=True)
df_hcs_C_epoch_first45=df_C_epoch_first45.where(df_C_epoch_first45['group']==0).dropna().reset_index(drop=True)
df_pts_I_epoch_first45=df_I_epoch_first45.where(df_I_epoch_first45['group']==1).dropna().reset_index(drop=True)
df_pts_C_epoch_first45=df_C_epoch_first45.where(df_C_epoch_first45['group']==1).dropna().reset_index(drop=True)

subj     model   variable       AIC    β_coef    p_vals   std_err  \
0  hc001  dACC ~ 1  Intercept  0.710214  0.050161  0.004829  0.017587   

     CI_low   CI_high  
0  0.015467  0.084855

subj          model   variable      AIC    β_coef    p_vals   std_err  \
0  hc001  dACC ~ rt + 1  Intercept -2.27544  0.050161  0.004411  0.017403   
1  hc001  dACC ~ rt + 1         rt -2.27544  0.038909  0.026551  0.017403   

     CI_low   CI_high  
0  0.015829  0.084493  
1  0.004577  0.073241

subj             model   variable       AIC    β_coef    p_vals   std_err  \
0  hc001  dACC ~ adapt + 1  Intercept  1.061253  4.588274  0.197182  3.545174   
1  hc001  dACC ~ adapt + 1      adapt  1.061253  3.820970  0.202097  2.984905   

     CI_low    CI_high  
0 -2.405402  11.581949  
1 -2.067446   9.709385

subj                model   variable       AIC    β_coef    p_vals  \
0  hc001  dACC ~ conflict + 1  Intercept  0.385906 -0.223444  0.217853   
1  hc001  dACC ~ conflict + 1   conflict  0.385906  0.681151  0.129910   

    std_err    CI_low   CI_high  
0  0.180718 -0.579952  0.133065  
1  0.447786 -0.202209  1.564511

subj                        model   variable       AIC    β_coef  \
0  hc001  dACC ~ adapt + conflict + 1  Intercept  2.194001  1.798750   
1  hc001  dACC ~ adapt + conflict + 1      adapt  2.194001  1.650792   
2  hc001  dACC ~ adapt + conflict + 1   conflict  2.194001  0.527869   

     p_vals   std_err    CI_low    CI_high  
0  0.699668  4.655569 -7.385757  10.983257  
1  0.664292  3.797638 -5.841189   9.142773  
2  0.356214  0.570728 -0.598063   1.653800

...

subj      model   variable         AIC    β_coef    p_vals   std_err  \
0  hc001  R_IFG ~ 1  Intercept  430.829202  0.058316  0.289283  0.054875   

     CI_low   CI_high  
0 -0.049935  0.166566

subj           model   variable         AIC    β_coef    p_vals   std_err  \
0  hc001  R_IFG ~ rt + 1  Intercept  426.715833  0.058316  0.282802  0.054139   
1  hc001  R_IFG ~ rt + 1         rt  426.715833  0.134234  0.014044  0.054139   

     CI_low   CI_high  
0 -0.048486  0.165118  
1  0.027432  0.241036

subj              model   variable         AIC     β_coef    p_vals  \
0  hc001  R_IFG ~ adapt + 1  Intercept  425.641731  29.405357  0.007623   
1  hc001  R_IFG ~ adapt + 1      adapt  425.641731  24.709424  0.007741   

     std_err    CI_low    CI_high  
0  10.900582  7.901442  50.909273  
1   9.177885  6.603924  42.814923

subj                 model   variable         AIC    β_coef  p_vals  \
0  hc001  R_IFG ~ conflict + 1  Intercept  432.673954  0.279579  0.6226   
1  hc001  R_IFG ~ conflict + 1   conflict  432.673954 -0.550846  0.6955   

    std_err    CI_low   CI_high  
0  0.567113 -0.839182  1.398341  
1  1.405198 -3.322924  2.221232

subj                         model   variable         AIC     β_coef  \
0  hc001  R_IFG ~ adapt + conflict + 1  Intercept  420.554426  53.724556   
1  hc001  R_IFG ~ adapt + conflict + 1      adapt  420.554426  43.629131   
2  hc001  R_IFG ~ adapt + conflict + 1   conflict  420.554426  -4.601982   

     p_vals    std_err     CI_low    CI_high  
0  0.000185  14.081161  25.945241  81.503871  
1  0.000197  11.486276  20.969004  66.289258  
2  0.008353   1.726214  -8.007457  -1.196507

In [16]:

# def results_summary_to_dataframe(results):
#     '''This takes the result of an statsmodel results table and transforms it into a dataframe'''
#     pvals = results.pvalues
#     coeff = results.params
#     conf_lower = results.conf_int()[0]
#     conf_higher = results.conf_int()[1]

#     results_df = pd.DataFrame({"pvals":pvals,
#                                "coeff":coeff,
#                                "conf_lower":conf_lower,
#                                "conf_higher":conf_higher
#                                 })
#     #Reordering...
#     results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
#     return results_df

##---------------------------------##

def OLS(equation,df_type,df_name,master_df):
    t_model = smf.ols(formula = '%s' % equation, data=df_type)
    t_lin_reg = t_model.fit()
    pvals = t_lin_reg.pvalues
    coeff = t_lin_reg.params
    stderr = t_lin_reg.bse
    AIC = t_lin_reg.aic
    conf_lower = t_lin_reg.conf_int(alpha=0.05,cols=None)[0]
    conf_higher = t_lin_reg.conf_int(alpha=0.05,cols=None)[1]
    results_df = pd.DataFrame(data={'model':equation,"subj":SUBJ,"p_vals":pvals,
                               "β_coef":coeff,"std_err":stderr,"CI_low":conf_lower,
                                "CI_high":conf_higher,"AIC":AIC})
    results_df.index.name = 'variable'
    results_df.reset_index(level=0, inplace=True)
    results_df = results_df[["subj","model","variable","AIC","β_coef","p_vals","std_err","CI_low","CI_high"]]  
    master_df=master_df.append(results_df)
    return master_df

results_df_ALL_subjs=pd.DataFrame(columns=["subj","model","variable","AIC",
                        "β_coef","p_vals","std_err","CI_low","CI_high"])

##---------------------------------##
model='dACC ~ conflict + 1'
OLS(model,df,'df',results_df_ALL_subjs)
##---------------------------------##
# dACC_pred_rt_ROIs_group_cond='dACC ~ group*conflict'
# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)

# dACC_pred_rt_ROIs_group_cond='dACC ~ group*conflict'
# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)

Empty DataFrame
Columns: [subj, model, variable, AIC, β_coef, p_vals, std_err, CI_low, CI_high]
Index: []

### Sanity check purposes only: Test an infinite amount of interactions

In [ ]:
# rt_pred_ROIs_group_cond='rt ~ cond + group + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_group_cond='dACC ~ cond + group + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_group_cond='L_IFG ~ cond + group + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_group_cond='R_IFG ~ cond + group + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_group_cond='L_dlPFC ~ cond + group + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_group_cond='R_dlPFC ~ cond + group + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_group_cond)
# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)
# OLS(L_IFG_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_group_cond)
# OLS(R_IFG_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_group_cond)
# OLS(L_dlPFC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_group_cond)
# OLS(R_dlPFC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_group_cond)

# ##---------------------------------##
# rt_pred_ROIs_cond='rt ~ cond + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_cond='dACC ~ cond + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_cond='L_IFG ~ cond + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_cond='R_IFG ~ cond + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_cond='L_dlPFC ~ cond + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_cond='R_dlPFC ~ cond + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)

# OLS(rt_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)

# OLS(rt_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)
# ##---------------------------------##
# rt_pred_ROIs_group='rt ~ group + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_group='dACC ~ group + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_group='L_IFG ~ group + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_group='R_IFG ~ group + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_group='L_dlPFC ~ group + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_group='R_dlPFC ~ group + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_group)

# OLS(rt_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_dlPFC_pred_rt_ROIs_group)

# OLS(rt_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_dlPFC_pred_rt_ROIs_group)
# ##---------------------------------##
# dACC_pred_ROIs_cond='dACC ~ cond + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_ROIs_cond='L_IFG ~ cond + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_ROIs_cond='R_IFG ~ cond + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_ROIs_cond='L_dlPFC ~ cond + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_ROIs_cond='R_dlPFC ~ cond + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(dACC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)

# OLS(dACC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)

# OLS(dACC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)
# ##---------------------------------##
# dACC_pred_rt_ROIs_group='dACC ~ group + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_ROIs_group='L_IFG ~ group + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_ROIs_group='R_IFG ~ group + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_ROIs_group='L_dlPFC ~ group + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_ROIs_group='R_dlPFC ~ group + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(dACC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_ROIs_group)

# OLS(dACC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_dlPFC_pred_ROIs_group)

# OLS(dACC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_dlPFC_pred_ROIs_group)
# ##---------------------------------##
# rt_pred_ROIs='rt ~ dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs='dACC ~ rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs='L_IFG ~ rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs='R_IFG ~ rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs='L_dlPFC ~ rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs='R_dlPFC ~ rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs,'CTRLS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_pts,'PTS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# ##---------------------------------##
# dACC_pred_rt_ROIs_group_cond='dACC ~ cond + rt + group'
# L_IFG_pred_rt_ROIs_group_cond='L_IFG ~ cond + group + rt'
# R_IFG_pred_rt_ROIs_group_cond='R_IFG ~ cond + group + rt'
# L_dlPFC_pred_rt_ROIs_group_cond='L_dlPFC ~ cond + group + rt'
# R_dlPFC_pred_rt_ROIs_group_cond='R_dlPFC ~ cond + group + rt'

# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# ##---------------------------------##
# rt_pred_group_cond='rt ~ cond + group'
# dACC_pred_group_cond='dACC ~ cond + group'
# L_IFG_pred_group_cond='L_IFG ~ cond + group'
# R_IFG_pred_group_cond='R_IFG ~ cond + group'
# L_dlPFC_pred_group_cond='L_dlPFC ~ cond + group'
# R_dlPFC_pred_group_cond='R_dlPFC ~ cond + group'

# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# ##---------------------------------##
# dACC_pred_rt='dACC ~ rt'
# L_IFG_pred_rt='L_IFG ~ rt'
# R_IFG_pred_rt='R_IFG ~ rt'
# L_dlPFC_pred_rt='L_dlPFC ~ rt'
# R_dlPFC_pred_rt='R_dlPFC ~ rt'

# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)

# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)

# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)

# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# ##---------------------------------##
# rt_pred_cond='rt ~ cond'
# dACC_pred_cond='dACC ~ cond'
# L_IFG_pred_cond='L_IFG ~ cond'
# R_IFG_pred_cond='R_IFG ~ cond'
# L_dlPFC_pred_cond='L_dlPFC ~ cond'
# R_dlPFC_pred_cond='R_dlPFC ~ cond'

# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')

# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')

# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# ##---------------------------------##
# rt_pred_group='rt ~ group'
# dACC_pred_group='dACC ~ group'
# L_IFG_pred_group='L_IFG ~ group'
# R_IFG_pred_group='R_IFG ~ group'
# L_dlPFC_pred_group='L_dlPFC ~ group'
# R_dlPFC_pred_group='R_dlPFC ~ group'

# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')

# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')

# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# ##---------------------------------##